In [62]:
import numpy as np
import matplotlib as plt
from tabulate import tabulate

### Resolver o sistema:

 2x + y + 3z = 4

 -1x + 3y + 2z = 3

 3x + y - 3z = 2

In [63]:
A = np.array([[2, 1, 3], [-1, 3, 2], [3, 1, -3]])
ba = np.array([4, 3, 2])
Ai = np.linalg.inv(A)
r = Ai @ ba
r

array([0.81632653, 0.95918367, 0.46938776])

In [64]:
def cofat(m):
    n = len(m)
    cof = np.zeros((n, n))
    for col in range(n):
        for lin in range(n):
            d_ij = np.delete(m, lin, 0)
            d_ij = np.delete(d_ij, col, 1)
            det = np.linalg.det(d_ij)
            cof[lin][col] = (-1) ** (lin + col) * det
    return cof


def adjuta(m):
    return cofat(m).transpose()


def inv_cofat(m):
    return (1 / np.linalg.det(m)) * adjuta(m)

In [65]:
A = np.array([[2, 1, 3], [-1, 3, 2], [3, 1, -3]])
Ai1 = np.linalg.inv(A)
Ai2 = inv_cofat(A)
print("Ai com linalg:\n", Ai1, "\nAi com inv_cofat:\n", Ai2)

Ai com linalg:
 [[ 0.2244898  -0.12244898  0.14285714]
 [-0.06122449  0.30612245  0.14285714]
 [ 0.20408163 -0.02040816 -0.14285714]] 
Ai com inv_cofat:
 [[ 0.2244898  -0.12244898  0.14285714]
 [-0.06122449  0.30612245  0.14285714]
 [ 0.20408163 -0.02040816 -0.14285714]]


In [66]:
A = np.array([[2, 1, 3], [-1, 3, 2], [3, 1, -3]])
ba = np.array(([4, 3, 2]))
x1 = np.linalg.solve(A, ba)
x2 = inv_cofat(A) @ ba
print(x1, " = ", x2)

[0.81632653 0.95918367 0.46938776]  =  [0.81632653 0.95918367 0.46938776]


In [67]:
def det_cofat(M):
    n = len(M)
    if n == 1:
        return M
    cof = np.zeros(n)
    for col in range(n):
        d = np.delete(M, 0, 0)
        d = np.delete(d, col, 1)
        cof[col] = ((-1) ** col) * det_cofat(d)
    return M[0, :] @ cof

In [68]:
np.linalg.det(A), det_cofat(A)

(-49.000000000000014, -49.0)

In [69]:
def cramer(M, B, prt):
    n = len(M)
    c = np.c_[M, B]
    d = det_cofat(M)
    x = np.zeros(n)
    if prt:
        print("Matriz aumentada [C=A|b] det(A) = %f" % d)
        print(c)
    for k in range(0, n):
        ak = M.copy()
        ak[:, k] = B
        dk = det_cofat(ak)
        x[k] = dk/d
        if prt:
            print("Matriz A%d    det(A%1d) = %f" % (k, k, dk))
            print(ak)
            print("x(%d) = %f / %f = %f\n" % (k, dk, d, x[k]))
    return x

In [70]:
inv_cofat(A)@ba

array([0.81632653, 0.95918367, 0.46938776])

In [71]:
cramer(A, ba, True)

Matriz aumentada [C=A|b] det(A) = -49.000000
[[ 2  1  3  4]
 [-1  3  2  3]
 [ 3  1 -3  2]]
Matriz A0    det(A0) = -40.000000
[[ 4  1  3]
 [ 3  3  2]
 [ 2  1 -3]]
x(0) = -40.000000 / -49.000000 = 0.816327

Matriz A1    det(A1) = -47.000000
[[ 2  4  3]
 [-1  3  2]
 [ 3  2 -3]]
x(1) = -47.000000 / -49.000000 = 0.959184

Matriz A2    det(A2) = -23.000000
[[ 2  1  4]
 [-1  3  3]
 [ 3  1  2]]
x(2) = -23.000000 / -49.000000 = 0.469388



array([0.81632653, 0.95918367, 0.46938776])

In [72]:
def eliminar_linha(lin, p, c, prt):
  c = c.astype(float)
  [N, M] = c.shape
  if prt:
    print("(L%d)=(L%d)-(%f)/(%f)*L(%d)" % (lin, lin, c[lin, p], c[p, p], p))
  m = c[lin, p] / c[p, p]
  c[lin, p:M+1]=c[lin,p:M+1]-m*c[p,p:M+1]
  return c, m

def pivotar_coluna(p,l_ini,l_fim,c,prt):
     [N,M]=c.shape
     linhas = np.arange(l_ini,l_fim)
     dist=np.argmax(abs(c[linhas,p]))
     if dist!=0:  #troca linhas
         c[[ p, (dist+p)],:] = c[[ (dist+p) , p ],:]
         if prt:
             print("Trocando linhas %d e %d" % (p,dist+p))
             pdtabulate=lambda c:tabulate(c,headers='keys')
             print(pdtabulate(c))
     return c, dist

def substituicao_regressiva(a, b, prt):
  [N,M]=a.shape
  x = np.zeros(N)*np.nan
  if a[N - 1, N - 1]!=0: x[N - 1]= b[N - 1] / a[N - 1, N - 1];
  if prt:
      print("Substituição regressiva\nx(%d)=%f" % (N-1,x[N-1]))
  for lin in range(N-2,-1,-1):
     if a[lin, lin]!=0:
       x[lin]=(b[lin]-a[lin,lin+1:N] @ x[lin+1:N])/a[lin,lin]
     if prt:
         print("x(%d)=%f" % (lin,x[lin]))
  return x

def substituicao_progressiva(a, b, prt):
  [N,M] = a.shape
  y = np.zeros(N)
  y[0]=b[0]
  if prt:
      print("Substituição Progressiva\ny(%d)=%f" % (1,y(1)))
  for lin in range(1,N):
        y[lin]= b[lin]-a[lin,0:lin] @ y[0:lin]
        if prt:
            print("y(%d)=%f" % (lin,y[lin]))
  return y

In [73]:
def eliminacao_gauss(a, b, prt, pivot): # com pivotamento
   a = a.astype(float)
   b = b.astype(float)
   C = np.c_[a, b]
   [N, M] = C.shape
   x = np.zeros(N)
   if prt:
       print("Matriz Aumentada [C=A|b]")
       print(C)
   for p in range(N-1):
       if pivot:
         (C,dist) = pivotar_coluna(p,p,N,C,prt)
       elif C[p, p] == 0:
          break
       if prt:
            print("Eliminando coluna %d com Pivô %f" % (p,C[p,p]) )
       for lin in range(p+1,N): #eliminação progressiva
           (C,m) = eliminar_linha(lin,p,C,prt)
       if prt:
           print(C)
   if C[p, p] != 0:
         x = substituicao_regressiva(C[:,0:N],C[:,N],prt)
   else:
         print("Não há solução única pois matriz A é singular")
         x[1:N] = np.inf
   return x

In [74]:
A, ba

(array([[ 2,  1,  3],
        [-1,  3,  2],
        [ 3,  1, -3]]),
 array([4, 3, 2]))

In [75]:
eliminacao_gauss(A, ba, True, False)

Matriz Aumentada [C=A|b]
[[ 2.  1.  3.  4.]
 [-1.  3.  2.  3.]
 [ 3.  1. -3.  2.]]
Eliminando coluna 0 com Pivô 2.000000
(L1)=(L1)-(-1.000000)/(2.000000)*L(0)
(L2)=(L2)-(3.000000)/(2.000000)*L(0)
[[ 2.   1.   3.   4. ]
 [ 0.   3.5  3.5  5. ]
 [ 0.  -0.5 -7.5 -4. ]]
Eliminando coluna 1 com Pivô 3.500000
(L2)=(L2)-(-0.500000)/(3.500000)*L(1)
[[ 2.          1.          3.          4.        ]
 [ 0.          3.5         3.5         5.        ]
 [ 0.          0.         -7.         -3.28571429]]
Substituição regressiva
x(2)=0.469388
x(1)=0.959184
x(0)=0.816327


array([0.81632653, 0.95918367, 0.46938776])

##### Pivoteamento consiste em trocar as linhas de modo que o elemento pivot (diag. principal) seja sempre o maior da coluna. Resulta em menor propagação de erros de arredondamento.

In [76]:
eliminacao_gauss(A, ba, True, True)

Matriz Aumentada [C=A|b]
[[ 2.  1.  3.  4.]
 [-1.  3.  2.  3.]
 [ 3.  1. -3.  2.]]
Trocando linhas 0 e 2
  0    1    2    3
---  ---  ---  ---
  3    1   -3    2
 -1    3    2    3
  2    1    3    4
Eliminando coluna 0 com Pivô 3.000000
(L1)=(L1)-(-1.000000)/(3.000000)*L(0)
(L2)=(L2)-(2.000000)/(3.000000)*L(0)
[[ 3.          1.         -3.          2.        ]
 [ 0.          3.33333333  1.          3.66666667]
 [ 0.          0.33333333  5.          2.66666667]]
Eliminando coluna 1 com Pivô 3.333333
(L2)=(L2)-(0.333333)/(3.333333)*L(1)
[[ 3.          1.         -3.          2.        ]
 [ 0.          3.33333333  1.          3.66666667]
 [ 0.          0.          4.9         2.3       ]]
Substituição regressiva
x(2)=0.469388
x(1)=0.959184
x(0)=0.816327


array([0.81632653, 0.95918367, 0.46938776])

#### def: o determinante de uma matriz triangular superior é o produto dos elementos de sua diagonal principal

#### podemos então calcular o determinante através da Eliminação Progressiva de Gauss, que transforma a matriz A em uma matriz triangular superior.

In [77]:
def det_gauss(a, prt):
    a = a.astype(float)
    [N, M] = a.shape
    if prt:
        print("Matriz A", A)
    n_pivot = 0
    for p in range(N-1):
        (a, dist) = pivotar_coluna(p, p, N, a, prt)
        if dist > 0:
            n_pivot += 1
        elif a[p, p] == 0:
            break
        if prt:
            print("Eliminando coluna %d com pivot %f\n" % (p, a[p, p]))
        for lin in range(p+1, N):
            (a, m) = eliminar_linha(lin, p, a, prt)
        if prt:
            print(a)
    d = np.prod(np.diag(a) * (-1)**n_pivot)
    return d

In [78]:
det_gauss(A, True), det_cofat(A)

Matriz A [[ 2  1  3]
 [-1  3  2]
 [ 3  1 -3]]
Trocando linhas 0 e 2
  0    1    2
---  ---  ---
  3    1   -3
 -1    3    2
  2    1    3
Eliminando coluna 0 com pivot 3.000000

(L1)=(L1)-(-1.000000)/(3.000000)*L(0)
(L2)=(L2)-(2.000000)/(3.000000)*L(0)
[[ 3.          1.         -3.        ]
 [ 0.          3.33333333  1.        ]
 [ 0.          0.33333333  5.        ]]
Eliminando coluna 1 com pivot 3.333333

(L2)=(L2)-(0.333333)/(3.333333)*L(1)
[[ 3.          1.         -3.        ]
 [ 0.          3.33333333  1.        ]
 [ 0.          0.          4.9       ]]


(-49.0, -49.0)

### Gauss-Jordan

In [79]:
def eliminacao_gaussjordan(a, b, prt):
   a = a.astype(float)
   b = b.astype(float)
   C = np.c_[a, b]
   [N, M] = C.shape
   x = np.zeros(N)
   if prt:
       print("Matriz Aumentada [C=A|b]")
       print(C)
   for p in range(N):
       (C, dist) = pivotar_coluna(p,p,N,C,prt)
       C[p, :] = C[p, :] / C[p, p]
       if prt:
            print("Eliminando coluna %d com Pivô %f" % (p,C[p,p]) )
       linhas = np.concatenate((np.arange(0, p), np.arange(p+1, N)))
       for lin in linhas:
           (C, m) = eliminar_linha(lin,p,C,prt)
       if prt:
           print(C)
   if C[p, p] != 0:
         x = C[:, N]
   else:
         print("Não há solução única pois matriz A é singular")
         x[1:N] = np.inf
   return x

In [80]:
eliminacao_gaussjordan(A, ba, True)

Matriz Aumentada [C=A|b]
[[ 2.  1.  3.  4.]
 [-1.  3.  2.  3.]
 [ 3.  1. -3.  2.]]
Trocando linhas 0 e 2
  0    1    2    3
---  ---  ---  ---
  3    1   -3    2
 -1    3    2    3
  2    1    3    4
Eliminando coluna 0 com Pivô 1.000000
(L1)=(L1)-(-1.000000)/(1.000000)*L(0)
(L2)=(L2)-(2.000000)/(1.000000)*L(0)
[[ 1.          0.33333333 -1.          0.66666667]
 [ 0.          3.33333333  1.          3.66666667]
 [ 0.          0.33333333  5.          2.66666667]]
Eliminando coluna 1 com Pivô 1.000000
(L0)=(L0)-(0.333333)/(1.000000)*L(1)
(L2)=(L2)-(0.333333)/(1.000000)*L(1)
[[ 1.   0.  -1.1  0.3]
 [ 0.   1.   0.3  1.1]
 [ 0.   0.   4.9  2.3]]
Eliminando coluna 2 com Pivô 1.000000
(L0)=(L0)-(-1.100000)/(1.000000)*L(2)
(L1)=(L1)-(0.300000)/(1.000000)*L(2)
[[1.         0.         0.         0.81632653]
 [0.         1.         0.         0.95918367]
 [0.         0.         1.         0.46938776]]


array([0.81632653, 0.95918367, 0.46938776])

##### Para calcular matriz inversa por Gauss-Jordan, primeiro criamos a matriz aumentada com A na esquerda e Identidade (I) na direita, e manipulamos (eliminacao gauss) tal matriz até termos I na esquerda, e por consequencia, A^-1 na direita.

In [81]:
def inv_gauss(a, prt):
    a = a.astype(float)
    [N, M] = a.shape
    C = np.c_[a, np.eye(N)]
    [N, M] = C.shape
    _Ai = np.zeros((N, N))
    if prt:
        print("Matriz Aumentada C = [A|Ai] ")
        print(C)
    for p in range(N):
        (C, dist) = pivotar_coluna(p, p, N, C, prt)
        C[p, :] = C[p, :] / C[p, p]
        if prt:
            print("Eliminando coluna %d com Pivô %f" % (p,C[p,p]) )
            linhas = np.concatenate((np.arange(0, p), np.arange(p+1, N)))
            for lin in linhas:
                (C, m) = eliminar_linha(lin, p, C, prt)
                if prt:
                    print(C)
                if C[p, p] != 0:
                    _Ai = C[0:N, N:2*N]
                else:
                    print("não há solução única pois matriz A é singular.")
                    _Ai = []
    return _Ai

In [82]:
inv_cofat(A)

array([[ 0.2244898 , -0.12244898,  0.14285714],
       [-0.06122449,  0.30612245,  0.14285714],
       [ 0.20408163, -0.02040816, -0.14285714]])

In [83]:
inv_gauss(A, True)

Matriz Aumentada C = [A|Ai] 
[[ 2.  1.  3.  1.  0.  0.]
 [-1.  3.  2.  0.  1.  0.]
 [ 3.  1. -3.  0.  0.  1.]]
Trocando linhas 0 e 2
  0    1    2    3    4    5
---  ---  ---  ---  ---  ---
  3    1   -3    0    0    1
 -1    3    2    0    1    0
  2    1    3    1    0    0
Eliminando coluna 0 com Pivô 1.000000
(L1)=(L1)-(-1.000000)/(1.000000)*L(0)
[[ 1.          0.33333333 -1.          0.          0.          0.33333333]
 [ 0.          3.33333333  1.          0.          1.          0.33333333]
 [ 2.          1.          3.          1.          0.          0.        ]]
(L2)=(L2)-(2.000000)/(1.000000)*L(0)
[[ 1.          0.33333333 -1.          0.          0.          0.33333333]
 [ 0.          3.33333333  1.          0.          1.          0.33333333]
 [ 0.          0.33333333  5.          1.          0.         -0.66666667]]
Eliminando coluna 1 com Pivô 1.000000
(L0)=(L0)-(0.333333)/(1.000000)*L(1)
[[ 1.          0.         -1.1         0.         -0.1         0.3       ]
 [ 0.  

array([[ 0.2244898 , -0.12244898,  0.14285714],
       [-0.06122449,  0.30612245,  0.14285714],
       [ 0.20408163, -0.02040816, -0.14285714]])

#### Fatoração LU

In [84]:
def fatoracao_lu(a,prt): # com pivotamento
   a = a.astype(float)
   [N,M]=A.shape
   L = np.eye(N)
   P = L.copy()
   U = A.copy()
   if prt:
       print("Matriz Aumentada [C=A|b]")
       print(np.c_[L, U])
   for p in range(N-1):
       (U ,dist)=pivotar_coluna(p,p,N,U,prt)
       if dist>1:
          P[[ p, (dist+p)],:] = P[[ (dist+p) , p ],:]
          for col in range(p): # troca linhas abaixo da diagonal
            if col<p:
               L[[p,(dist+p)],col] = L[[(dist+p),p],col]
          if prt:
            print("Trocando linhas %d e %d de U" % (p,p+dist))
            print("Trocando linhas %d e %d de L abaixo da diagonal" % (p,p+dist))
            print(np.c_[L, U])
       if U[p, p]==0:
           break
       for lin in range(p+1,N): #eliminação progressiva
           (U, m)=eliminar_linha(lin,p,U,prt)
           L[lin,p] = m
       if prt:
           print(np.c_[L, U])
   return L, U, P


#### Resolução de sistema usando os fatores L, U

In [85]:
def solucao_lu(L,U,P,b,prt):
    bp = P @ b
    y = substituicao_progressiva(L,bp,False)
    x = substituicao_regressiva(U,y,False)
    if prt:
        print("[b bp]")
        print(np.c_[b, bp])
        print("matriz aumentada  [L  bp y]")
        print(np.c_[L, bp, y])
        print("matriz aumentada  [U y x]")
        print(np.c_[U, y, x])
    return x

In [86]:
(l, u, p_) = fatoracao_lu(A, True)
(l, u, p_)

Matriz Aumentada [C=A|b]
[[ 1.  0.  0.  2.  1.  3.]
 [ 0.  1.  0. -1.  3.  2.]
 [ 0.  0.  1.  3.  1. -3.]]
Trocando linhas 0 e 2
  0    1    2
---  ---  ---
  3    1   -3
 -1    3    2
  2    1    3
Trocando linhas 0 e 2 de U
Trocando linhas 0 e 2 de L abaixo da diagonal
[[ 1.  0.  0.  3.  1. -3.]
 [ 0.  1.  0. -1.  3.  2.]
 [ 0.  0.  1.  2.  1.  3.]]
(L1)=(L1)-(-1.000000)/(3.000000)*L(0)
(L2)=(L2)-(2.000000)/(3.000000)*L(0)
[[ 1.          0.          0.          3.          1.         -3.        ]
 [-0.33333333  1.          0.          0.          3.33333333  1.        ]
 [ 0.66666667  0.          1.          0.          0.33333333  5.        ]]
(L2)=(L2)-(0.333333)/(3.333333)*L(1)
[[ 1.          0.          0.          3.          1.         -3.        ]
 [-0.33333333  1.          0.          0.          3.33333333  1.        ]
 [ 0.66666667  0.1         1.          0.          0.          4.9       ]]


(array([[ 1.        ,  0.        ,  0.        ],
        [-0.33333333,  1.        ,  0.        ],
        [ 0.66666667,  0.1       ,  1.        ]]),
 array([[ 3.        ,  1.        , -3.        ],
        [ 0.        ,  3.33333333,  1.        ],
        [ 0.        ,  0.        ,  4.9       ]]),
 array([[0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.]]))

In [87]:
solucao_lu(l, u, p_, ba, True)

[b bp]
[[4. 2.]
 [3. 3.]
 [2. 4.]]
matriz aumentada  [L  bp y]
[[ 1.          0.          0.          2.          2.        ]
 [-0.33333333  1.          0.          3.          3.66666667]
 [ 0.66666667  0.1         1.          4.          2.3       ]]
matriz aumentada  [U y x]
[[ 3.          1.         -3.          2.          0.81632653]
 [ 0.          3.33333333  1.          3.66666667  0.95918367]
 [ 0.          0.          4.9         2.3         0.46938776]]


array([0.81632653, 0.95918367, 0.46938776])

#### Sistemas tridiagonais

In [88]:
def tridiagonal(dl,dp,du,r):
    dl= dl.astype(float)
    dp= dp.astype(float)
    du= du.astype(float)
    r = r.astype(float)
    N=len(r)
    y = np.zeros(N)
    for k in range(1,N): #eliminação progressiva
        m=dl[k-1]/dp[k-1];
        dp[k]=dp[k]-m*du[k-1];
        r[k]=r[k]-m*r[k-1];
    y[N-1]=r[N-1]/dp[N-1]; #substituição regressiva
    for k in range(N-2,-1,-1):
        y[k]=(r[k]-du[k]*y[k+1])/dp[k];
    return y

In [89]:
dl = np.array([7,9,4,1,2,34,32,65,23,12,12,5,9,3])
dp = np.array([1,3,5,3,4,6,7,5,3,5,6,8,10,7,14])
du = np.array([3,7,3,2,1,5,3,9,8,24,12,6,7,8])
r=np.array([1,1.98,4.99,2.98,6.97,4.01,4.98,2.25,3.37,6.94,4.92,3.04,5.97,7.91,1.89]) # vetor dos termos independentes
A = np.diag(dp) + np.diag(du, 1) + np.diag(dl, -1)
A

array([[ 1,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 7,  3,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  9,  5,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  4,  3,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  1,  4,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  2,  6,  5,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 34,  7,  3,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, 32,  5,  9,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 65,  3,  8,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 23,  5, 24,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 12,  6, 12,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 12,  8,  6,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5, 10,  7,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9,  7,  8],
       [ 0,  0,  0,  0,  0,  0,  0

In [90]:
tridiagonal(dl, dp, du, r)

array([-0.77,  0.59,  0.8 , -1.44,  2.05,  0.21, -0.27, -0.09,  1.26,
        0.68, -1.06,  0.26,  2.28, -2.59,  0.69])

### Métodos Iterativos p/ resolução de sistemas lineares

In [101]:
def gaussjacobi(a,b,x_ini,tol,prt):
     N = len(b)
     erro_lista=[]
     erro=np.nan
     erro_lista.append(erro)
     x1 = np.zeros(N)
     x0 = np.array(x_ini,dtype="float64")
     if prt:
        print("Matriz Aumentada [A,b]")
        print(np.c_[a, b])
        print("k\t\t[x]\t\t\terro")
        np.set_printoptions(precision=4)
        print(str(0)+' ['+', '.join('{:.8f}'.format(i) for i in x0)+'] ' +'{:.1e}'.format(erro))
     for k in range (1,500):
        for j in range (N):
            x1[j]=(b[j]- a[j,:]@x0 + a[j,j]*x0[j])/a[j,j]
        if np.linalg.norm(x1) != 0:
            erro=np.linalg.norm(x1-x0)/np.linalg.norm(x1)
        x0[:]=x1[:]
        #x0=x1
        if prt:
            print(str(k)+' ['+', '.join('{:.8f}'.format(i) for i in x0)+'] ' +'{:.1e}'.format(erro))
        erro_lista.append(erro)
        if erro<tol:
            break
     if prt:
        print("\nSolução x=",' [',', '.join('{:.8f}'.format(i) for i in x0),']')
        print ('após %i iterações com erro=%.1e\n' % (k,erro))
     return x1, erro_lista

In [103]:
A = np.array([[2, 1, 3], [-1, 3, 2], [3, 1, -3]])
A, ba

(array([[ 2,  1,  3],
        [-1,  3,  2],
        [ 3,  1, -3]]),
 array([4, 3, 2]))

In [110]:
A2 = np.array([[8,2,4], [1,11,3], [5,3,10]])
b2 = np.array([9,18,72])
(xr, err) = gaussjacobi(A2, b2, [0, 0, 0], 1e-6, True)

Matriz Aumentada [A,b]
[[ 8  2  4  9]
 [ 1 11  3 18]
 [ 5  3 10 72]]
k		[x]			erro
0 [0.00000000, 0.00000000, 0.00000000] nan
1 [1.12500000, 1.63636364, 7.20000000] 1.0e+00
2 [-2.88409091, -0.42954545, 6.14659091] 6.8e-01
3 [-1.84090909, 0.22221074, 8.77090909] 3.2e-01
4 [-3.31600723, -0.58834711, 8.05379132] 2.1e-01
5 [-2.75480888, -0.25866970, 9.03450775] 1.2e-01
6 [-3.32758645, -0.57715585, 8.65500535] 8.2e-02
7 [-3.05821371, -0.42158451, 9.03693998] 5.2e-02
8 [-3.28807386, -0.55023693, 8.85558221] 3.4e-02
9 [-3.16523187, -0.47987934, 9.00910801] 2.2e-02
10 [-3.25958417, -0.53291747, 8.92657974] 1.4e-02
11 [-3.20506050, -0.50183228, 8.98966733] 9.3e-03
12 [-3.24437559, -0.52399468, 8.95307993] 6.1e-03
13 [-3.22054130, -0.51044220, 8.97938620] 4.0e-03
14 [-3.23708255, -0.51978339, 8.96340331] 2.6e-03
15 [-3.22675581, -0.51392067, 8.97447629] 1.7e-03
16 [-3.23375798, -0.51787937, 8.96755410] 1.1e-03
17 [-3.22930721, -0.51535494, 8.97224280] 7.3e-04
18 [-3.23228267, -0.51703829, 8.9692

In [111]:
xr

array([-3.2311, -0.5164,  8.9705])